# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd

In [9]:
driver = webdriver.Chrome()
url = "https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C"
driver.get(url)


## Search for the TLDR Number `006179570C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [10]:
button1= driver.find_element_by_xpath('//*[@id="mcrbutton"]')
driver.execute_script("arguments[0].scrollIntoView(true)", button1)
button1.click()

In [11]:
text_input= driver.find_element_by_xpath('//*[@id="mcrdata"]')
driver.execute_script("arguments[0].scrollIntoView(true)", text_input)
text_input.send_keys('006179570C')

In [12]:
button2 = driver.find_element_by_xpath('//*[@id="submit3"]/b')
button2.click()

# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

In [13]:
#The business name
#Owner/operator
#Phone number
#License status (Active, Expired, Etc)
#Physical address

In [14]:
b_name = driver.find_elements_by_tag_name('td')[5].text.replace("Name:","")
print(b_name)

owner= driver.find_elements_by_tag_name('td')[7].text.replace("Owner/Officer:","")
print(owner)

phone_number = driver.find_elements_by_tag_name('td')[9].text.replace("Phone:","")
print(phone_number)

license = driver.find_elements_by_tag_name('tr')[7].text.replace("Certificate Information: Status:","")
print(license)

address= driver.find_elements_by_tag_name('tr')[8].find_elements_by_tag_name('td')[1].text[-55:].replace("Physical:","")
#//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]/text()[5]
print(address)

Company = {
    'name': b_name,
    'owner':owner,
    'address': address,
    'phone_number': phone_number,
    'license': license
    
}

Company

   B.D. SMITH TOWING
   BRANDT SMITH / OWNER
   8173330706
  Expired

13619 BRETT JACKSON RD.
FORT WORTH, TX. 76179


{'name': '   B.D. SMITH TOWING',
 'owner': '   BRANDT SMITH / OWNER',
 'address': '\n13619 BRETT JACKSON RD.\nFORT WORTH, TX. 76179',
 'phone_number': '   8173330706',
 'license': '  Expired'}

# Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.

### Open up `trucks-subset.csv` and save it into a dataframe

In [15]:
df = pd.read_csv("trucks-subset.csv")

In [16]:
df.head(3)

,TDLR Number
0,006507931C
1,006179570C
2,006502097C


In [17]:
#https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C

## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C`.

- *TIP: Use .apply and a function*
- *TIP: Unlike the Yelp example, you'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', -1)` to display aaaalll of the text in a cell*

In [18]:
pd.set_option('display.max_colwidth', -1)

def get_url(row):
    return ("https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=" + row)
   
    

In [19]:
get_url(df)


,TDLR Number
0,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

In [20]:
df['url']= get_url(df)


In [21]:
df

,TDLR Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Go through each row of the dataset, printing out information about each tow truck company.

Now will be **scraping** inside of your function.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

Just print it out for now.

- *TIP: use .apply*
- *TIP: You'll be using the code you wrote before, but converted into a function*
- *TIP: Remember how the TDLR Number is in the URL? You don't need to do the form submission if you don't want!*
- *TIP: Make sure you adjust any variables so you don't scrape the same page again and again*

In [27]:
def find_trucks(row):
   for urls in url:
        driver.get(df['url'])
        driver.get(df['url'])

        b_name = driver.find_elements_by_tag_name('td')[5].text.replace("Name:","")
        return(b_name)

        owner= driver.find_elements_by_tag_name('td')[7].text.replace("Owner/Officer:","")
        return(owner)

        phone_number = driver.find_elements_by_tag_name('td')[9].text.replace("Phone:","")
        return(phone_number)

        license = driver.find_elements_by_tag_name('tr')[7].text.replace("Certificate Information: Status:","")
        return(license)

        address= driver.find_elements_by_tag_name('tr')[8].find_elements_by_tag_name('td')[1].text[-55:].replace("Physical:","")
        #//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]/text()[5]
        return (address)

        return pd.Series({
            'name': b_name,
            'owner':owner,
            'address': address,
            'phone_number': phone_number,
            'license': license

        })

driver = webdriver.Chrome()

new_df = df.apply(find_trucks, axis=1).join(df)
new_df.head()

TypeError: ("Object of type 'Series' is not JSON serializable", 'occurred at index 0')

TypeError: ("Object of type 'Series' is not JSON serializable", 'occurred at index 0')

## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

### Save your dataframe as a CSV named `tow-trucks-extended.csv`

### Re-open your dataframe to confirm you didn't save any extra weird columns

## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**